# Задание 7

## 1. PropertyCreator (0.2 балла)

Напишите мета класс для создания свойств (property) класса из функций начинающихся с "set\_", "get\_" или "del_". Пример использования:
<code>
class TestPropertyCreator(metaclass=PropertyCreator):
    def \__init\__(self, lo):
        self.__x = None
        self.lo = lo

    def get_x(self):
        return self.__x

    def set_x(self, value):
        if value < self.lo:
            raise ValueError("Value must in condition: {} <= value".format(self.lo))
        self.__x = value
    
    def del_x(self):
        self.__x = "No more"

    pass


obj = TestPropertyCreator(5)
obj.x = 4
print(obj.x)
del (obj.x)
</code>

* Мета класс должен поддерживать наследование, в смысле создавать свойства у потомков.
* Должен поддерживать частичное описание свойств, т. е. например, описание одного метода get_val (без set_val и del_val).
* Поддерживать множественное использование одного свойства с одним именем в разных классах:

<code>
class A(metaclass=PropertyCreator):
    def get_x(self):
        return "x in class A"

class B(metaclass=PropertyCreator):
    def get_x(self):
        return "x in class B"

class C(metaclass=PropertyCreator):
    def set_x(self, value):
        self.value = "x in class C"
    def get_x(self):
        return self.value
</code>
* Должен уметь обрабатывать имен с несколькими подчеркиваниями "get_raw_text".

#### Решение

Создадим метакласс `PropertyCreator` который будет управлять поведением при создании класса. 

In [41]:
def add_prop_to_dict(properties: dict, prop_name: str, prop_type:str, prop_body):
    if (properties.get(prop_name) == None):
        properties[prop_name] = {}
    properties[prop_name][prop_type] = prop_body

class PropertyCreator(type):
        
    def __new__(cls, name, bases, attrs):
        
        properties = dict()      
    
        # attrs present as dict name and callable func
        for attr_name, attr_body in attrs.items():
            if (callable(attr_body)):
                prop_name = attr_name[4:]
                prop_type =  attr_name[:4]
                if (prop_type in ['get_', 'set_', 'del_']):
                    add_prop_to_dict(properties, prop_name, prop_type[:3], attr_body)
                    
        for prop_name in properties:
            attrs[prop_name] = property(properties[prop_name].get('get'), properties[prop_name].get('set'), 
                                       properties[prop_name].get('del'))               
        
        return super().__new__(cls, name, bases, attrs)  

_Замечание:_ Если требовалось чтобы методы set и get создавались(в случае их отсутсвия к классе), то это решение внизу ноута).

#### Протестируйем свое решение

In [42]:
def test_simple():
    class TestPropertyCreator(metaclass=PropertyCreator):
        
        def __init__(self, lo):
            self.__x = None
            self.lo = lo
            
            
        def get_x(self):
            return self.__x

        def set_x(self, value):
            if value < self.lo:
                raise ValueError("Value must in condition: {} <= value".format(self.lo))
            self.__x = value

        def del_x(self):
            self.__x = "No more"
            
    t = TestPropertyCreator(1)
    t.x = 5
    try:
        t.x = 0
    except ValueError as e:
        print(e)
    print(t.x)
    
    del t.x
    
test_simple()

Value must in condition: 1 <= value
5


In [43]:
def test_with_inheritance():
    class TestPropertyCreator(metaclass=PropertyCreator):
        pass

    class TestPropertyCreatorInheritance(TestPropertyCreator):
        def __init__(self):
            self._secret_list = []

        def get_x(self):
            self._secret_list.append("get")
            return 0

        def set_x(self, value):
            self._secret_list.append("set")
    
    
    t = TestPropertyCreatorInheritance()
    t.x = 7
    try:
        t.x = 1
    except ValueError as e:
        print(e)
    print(t.x)
    print(t._secret_list)
    
test_with_inheritance()

0
['set', 'set', 'get']


In [44]:
def test_partially_defined():
    class TestPropertyCreator(metaclass=PropertyCreator):
        def __init__(self):
            self._secret_list = []

        def get_x(self):
            self._secret_list.append("get")
            return 0

        def set_y(self, value):
            self._secret_list.append("set")
            self._y = value
            
    t = TestPropertyCreator()
    t.y = 1
    try:
        t.y = 5
    except ValueError as e:
        print(e)
        
    print(t.x)
    print(t._secret_list)
            
test_partially_defined()

0
['set', 'set', 'get']


In [45]:
def test_sanity():
    class TestPropertyCreator(metaclass=PropertyCreator):
        _text = 0
        def get_raw_text(self):
             return 'raw_text'

        def get_text(self):
             return self._text % 2

        def set_text(self, value):
            try:
                self._text = int(value)
            except ValueError:
                raise TypeError("unproper value for text: {}".format(value))
    
    t = TestPropertyCreator()
    t.text = 1
    print(t.text)
    print(t.raw_text)

test_sanity()

1
raw_text


In [46]:
def test_multiple_usages():
    class TestPropertyCreatorA(metaclass=PropertyCreator):
        def get_x(self):
            return 0
        
    class TestPropertyCreatorB(metaclass=PropertyCreator):
        def get_x(self):
            return 1
        
    class TestPropertyCreatorC(metaclass=PropertyCreator):
        def set_x(self, value):
            self.value = value + 1
            
        def get_x(self):
            return self.value

    t_A = TestPropertyCreatorA()
    t_B = TestPropertyCreatorB()
    t_C = TestPropertyCreatorC()
    
    t_C.x = 5
    print(t_A.x)
    print(t_B.x)
    print(t_C.x)

test_multiple_usages()

0
1
6


## 2. InstanceCountExeptioner (0.2 балла)
Напишите метакласс InstanceCountExeptioner, который будет следить за количеством экземпляров класса, использующих его. Количество задается через поле класса \_\_max_instane\_count\_\_. Т. е. число экземпляров каждого класса регулируется отдельно. Если в классе не указано поле \_\_max_instane\_count\_\_, то используйте заранее заданное число в метаклассе (любое). Пример:




```
class TestInstanceCountExeptionerB(metaclass=InstanceCountExeptioner):
    \_\_max_instane\_count\_\_ = 1
    def \__init\__(self, a):
        self.a = a

a_one = TestInstanceCountExeptionerA('one')
a_two = TestInstanceCountExeptionerA('two')
b_one = TestInstanceCountExeptionerB('one')
\# пока всё шло хорошо

\# а вот
a_three = TestInstanceCountExeptionerA('three')
\# выкенет исключение InstanceCountExeption (ваше собственное исключение)
```

#### Решение

In [19]:
# Опишите исключение InstanceCountExeption

class InstanceCountExeption(Exception):
    
    def __init__(self, *args, **kwargs):
        Exception.__init__(self, *args, **kwargs)

In [51]:
# Опишите мета класс InstanceCountExeptioner
from collections import Counter, defaultdict

class InstanceCountExeptioner(type):

    counter = Counter()
    max_instance_counts = defaultdict(int)
    
    def __new__(cls, name, bases, attrs):
        cls.max_instance_counts[name] = attrs.get('__max_instance_count__', 999)
        cls.counter[name] = 0
        
        return super().__new__(cls, name, bases, attrs)
    
    def __call__(cls, *args, **kwargs):
        name = cls.__name__
        if cls.counter[name] >= cls.max_instance_counts[name]:
            raise InstanceCountExeption("reach the limit")
            
        cls.counter[name] += 1
        return super().__call__(*args, **kwargs)

#### Протестируйте свое решение

In [21]:
class TestInstanceCountExeptionerA(metaclass=InstanceCountExeptioner):
    __max_instance_count__ = 2

    def __init__(self):
        self.a = 1

    def get(self):
        return self.a
    
class TestInstanceCountExeptionerB(metaclass=InstanceCountExeptioner):
    __max_instance_count__ = 3

    def __init__(self):
        self.b = 2

    def get(self):
        return self.b

In [22]:
b_one = TestInstanceCountExeptionerB()
b_two = TestInstanceCountExeptionerB()
b_three = TestInstanceCountExeptionerB()

In [23]:
def test_create():
    a_one = TestInstanceCountExeptionerA()
    b_one = TestInstanceCountExeptionerB()

    assert a_one.get() == 1, 'A1 bad'
    assert b_one.get() == 1, 'B1 bad'
    print("test_create DONE")


def test_fail_create_a():
    a_one = TestInstanceCountExeptionerA()
    a_two = TestInstanceCountExeptionerA()

    try:
         a_three = TestInstanceCountExeptionerA()
    except InstanceCountExeption as e:
        print('test_fail_create_a')
        
    assert a_two.get() == 2, 'A2 bad'


def test_fail_create_b():
    b_one = TestInstanceCountExeptionerB()
    b_two = TestInstanceCountExeptionerB()
    b_three = TestInstanceCountExeptionerB()

    try:
        b_four = TestInstanceCountExeptionerB()
    except InstanceCountExeption as e:
        print('too many TestInstanceCountExeptionerB object')
    
    assert b_three.get() == 3, 'B3 bad'

    
try:
     test_create()
except InstanceCountExeption as e:
    print('test_create DONE')


try:
     test_create()
except InstanceCountExeption as e:
    print('test_fail_create_a DONE')

try:
     test_fail_create_b()
except InstanceCountExeption as e:
    print('test_fail_create_b DONE')


test_create DONE
test_fail_create_a DONE
test_fail_create_b DONE


## 3. JSONClassCreator (0.6 баллов)
Напишите метакласс, который будет по json представлению строить новый класс и обратно. Класс должен уметь следующее:
* Поддерживать сохранение и получение магических функций класса.
* Поддерживать сохранение и получение обычных функций.
* Поддерживать сохранение полей со стандартными типами.
* Уберите из сохранения следующие поля и методы: ['\_\_dict\_\_', '\_\_weakref\_\_', '\_\_module\_\_', '\_\_init\_\_']
* У создаваемого класса должна быть функция to_json_str

Формат json строки должен быть следующий:

<code>
{
    "name": название класса,
    "bases": базовые классы,
    "methods": методы класса,
    "attrs": поля класса
}
</code>

Рекомендации:
* Для получения кода функций используйте модуль <a href="http://python-lab.ru/documentation/27/stdlib/inspect.html">inspect</a>.
* Для того, чтобы запустить код функций, можно использовать exec.
* Можно не исправлять ошибку типа OSError: could not get source code - возникает для функций, полученных с помощью exec.

In [ ]:
import json


class ParentTest1(object):
    pass

class ParentTest2(object):
    pass

class Test(ParentTest1, ParentTest2):
    """Тестовый класс"""

    val = [1, 2, 3]

    def f(self, x):
        print(x)
    
    def __repr__(self):
        return "Test(val={})".format(self.val)

    def __str__(self):
        return "Test(val={})".format(self.val)

    pass

In [ ]:
print(*json.loads(JSONClassCreator.to_json_str(Test)).items(), sep="\n")

In [ ]:
tmp = JSONClassCreator(JSONClassCreator.to_json_str(Test))

In [ ]:
tmp_obj = tmp()
tmp_obj, tmp_obj.f("hi"), tmp.val, tmp.__doc__

In [ ]:
tmp.__dict__

#### Решение

В методе `__new__` 'распрарсим' json и вызовем super от полученных аргументов.

Вызывов `globals()` позволяет перейти в глобальное пространство

Реализация `to_json_str`: 
Для получения атрибутов класса будем использовать метод `inspect.getmembers`, причем с помощью метода
`isroutine` мы оставим лишь нужные функции.

In [47]:
import inspect
import json


class JSONClassCreator(type):

    @staticmethod
    def get_global_field(field: str):
        return globals()[field]

    def __new__(mcls, json_str):
        dict_js = json.loads(json_str)

        name = dict_js['name']
        bases = tuple(JSONClassCreator.get_global_field(base) for base in dict_js['bases'])

        for method_source in dict_js['methods'].values():
            exec(method_source, globals())

        methods = {
            method_name: JSONClassCreator.get_global_field(method_name)
            for method_name, method_source in dict_js['methods'].items()
        }

        attrs = dict_js['attrs']
        attrs.update(methods)
        return super().__new__(mcls, name, bases, attrs)

    def to_json_str(cls):
        exclude = ['__dict__', '__weakref__', '__module__', '__init__']

        class_name = cls.__name__
        bases = [base.__name__ for base in cls.__bases__]

        attrs = {name: val for name, val in
                 cls.__dict__.items()
                 if not name in exclude and name is not '__class__' and not inspect.isroutine(val)}

        methods = {
            method_name: inspect.getsource(method_source).strip()
            for method_name, method_source in cls.__dict__.items() if
            method_name not in exclude and method_name not in attrs.keys()
        }

        return json.dumps({
            "name": class_name,
            "bases": bases,
            "methods": methods,
            "attrs": attrs
        })


#### Проверьте свое решение на примере

In [48]:
class ParentTest1(object):
    pass

class ParentTest2(object):
    pass

class Test(ParentTest1, ParentTest2):
    """Тестовый класс"""

    val = [1, 2, 3]

    def f(self, x):
        print(x)
    
    def __repr__(self):
        return "Test(val={})".format(self.val)

    def __str__(self):
        return "Test(val={})".format(self.val)

    pass

In [49]:
print(*json.loads(JSONClassCreator.to_json_str(Test)).items(), sep="\n")

('name', 'Test')
('bases', ['ParentTest1', 'ParentTest2'])
('methods', {'f': 'def f(self, x):\n        print(x)', '__repr__': 'def __repr__(self):\n        return "Test(val={})".format(self.val)', '__str__': 'def __str__(self):\n        return "Test(val={})".format(self.val)'})
('attrs', {'__doc__': 'Тестовый класс', 'val': [1, 2, 3]})


In [50]:
tmp = JSONClassCreator(JSONClassCreator.to_json_str(Test))
tmp.__dict__

mappingproxy({'__doc__': 'Тестовый класс',
              'val': [1, 2, 3],
              'f': <function __main__.f(self, x)>,
              '__repr__': <function __main__.__repr__(self)>,
              '__str__': <function __main__.__str__(self)>,
              '__module__': '__main__'})

# Решение где set и get доопределяются

НЕ надо смотреть!

In [ ]:
class PropertyCreator2(type):
        
    def __new__(cls, name, bases, attrs):

        methods = {key: value for key, value in attrs.items()
                   if callable(value)}

        __get_methods = {key[4:]: value for key, value in methods.items()
                         if key.startswith('get_')}

        __set_methods = {key[4:]: value for key, value in methods.items()
                         if key.startswith('set_')}

        __s_get_methods = set(__get_methods)
        __s_set_methods = set(__set_methods)
        __s_g_methods = __s_get_methods.intersection(__s_set_methods)

        for key in __s_get_methods:
            if key not in __s_g_methods:
                setattr(cls, key, property(fget=__get_methods[key]))

        for key in __s_set_methods:
            if key not in __s_g_methods:
                setattr(cls, key, property(fset=__set_methods[key]))

        for key in __s_g_methods:
            setattr(cls, key, property(fget=__get_methods[key],
                                       fset=__set_methods[key]))
               
        
        return super().__new__(cls, name, bases, attrs)  